Thomas Dougherty

Probability and Statistics for Computer Science


### Analysis of the On-Time Performance (OTP) of New Jersey Transit Commuter Rail<br>
On-time performance refers to the level of success of a service adhering to its schedule. In this example I'll be exploring NJT commuter rail from March 2018 to March 2020. On-time performance data can be used for schedule planning, passenger information systems, and comparison to weather data. 
    

 March 2018

In [11]:
import os
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from helper_functions import *


all_services = helper.combine_csvs("data\\services\\")
all_services = helper.format_services(all_services)
helper.combine_csvs('data\\')

weather = pd.read_csv('data\\weather.csv')
march_weather = helper.format_weather(weather,'March')
march_weather.head(10)
all_services.head(10)

PermissionError: [Errno 13] Permission denied: 'data\\services'

In [ ]:
 #create new dataframe with data broken down by line
# function to create df. figure out how to import pandas into helper file???
mar_2018 = all_services[(all_services['date'].dt.month == 3) & (all_services['date'].dt.year == 2018)]
njt_lines=all_services['line'].unique()
max_delays = pd.Series(helper.get_max_delay(all_services,njt_lines)).round(3)
avg_delays = pd.Series(helper.get_avg_delay(all_services,njt_lines)).round(3)
dates = pd.Series(helper.get_delay_date(all_services,njt_lines))
train_ids = pd.Series(helper.get_train_ids(all_services,njt_lines))

performance_by_line_march = helper.create_new_dataframe(njt_lines, max_delays, avg_delays, dates)


performance_by_line_march

### November 2018:

In [ ]:
#read CSV for March 2018
services_nov = pd.read_csv('data\\2018_11.csv')
#status column is not needed so that will be dropped
services_nov = helper.format_services(services_nov)
services_nov.dtypes
services_nov

njt_lines=services_nov['line'].unique()
max_delays = pd.Series(helper.get_max_delay(services_nov,njt_lines)).round(3)
avg_delays = pd.Series(helper.get_avg_delay(services_nov,njt_lines)).round(3)
dates = pd.Series(helper.get_delay_date(services_nov,njt_lines))
train_ids = pd.Series(helper.get_train_ids(services_nov,njt_lines))

performance_by_line_nov = helper.create_new_dataframe(njt_lines, max_delays, avg_delays, dates)

performance_by_line_nov

In [ ]:
######### weather data 
####
###
##

nov_weather = helper.format_weather(weather,'November')

mode_date = performance_by_line_nov['Date of Longest Delay'].mode()

mode_date = mode_date.reset_index(drop=True)

print(pd.to_datetime(mode_date))

print(mode_date)
#nov_weather.dtypes
nov_weather.loc[nov_weather['DATE'] == mode_date]
nov_weather.head(10)

